In [1]:
cd

/Users/hanna


In [2]:
cd Documents

/Users/hanna/Documents


In [3]:
cd "Data Science"

/Users/hanna/Documents/Data Science


In [4]:
ls

12-13.pdf                   data.xls
2-task.pdf                  data2.xls
4.pdf                       lesson3Data_Science.pdf
5-DataScience.pdf           lesson6_data_science.pdf
9-data.pdf                  my_phone.csv
Lesson_11.pdf               phone_correct.csv
Lesson_14.pdf               phone_float.csv
Lesson_8-9.pdf              presentation_data_prof.pdf
Lesson_8.pdf                python books/
archive.zip                 winequalityN.csv


### На вхід дані зібрані у файлі my_phone.csv.

### Потрібно виконати наступні завдання: 

### Скористатись датасетом з ДЗ 18 про мобільні телефони, натренувати нейромережу, яка передбачатиме ціновий показник для телефона (price_range). Потрібно використати вже оброблені та почищені дані, щоб отримати якомога кращий результат.

### Кількість епох, шарів, точність та оптимальну модель потрібно обрати самостійно (можна методом спроб та помилок), головне не навмання, а з поясненнями. (інакше бали буде знято) (5 балів)

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('phone_correct.csv')
df.head()

,Unnamed: 0,battery_power,bluetooth,weight,memory,n_cores,ram,dual_sim,wifi,pixel_height,pixel_width,price_range
0,1,1021,1,136,53,3,2631,1,0,905,1988,2.0
1,2,563,1,145,41,5,2603,1,0,1263,1716,2.0
2,3,615,1,131,10,6,2769,0,0,1216,1786,2.0
3,4,1821,1,141,44,2,1411,0,0,1208,1212,1.0
4,5,1859,0,164,22,1,1067,1,0,1004,1654,1.0


<font size = 4><mark>Щоб довго не мучитися, я експортувала вже опрацьований датафрейм із попередньої домашньої роботи та<br><br> імпортувала сюди. Ще пару штрихів і він буде готовий для роботи.</mark>

In [ ]:
df['price_range'] = df['price_range'].astype('Int64')

In [32]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [33]:
df_full = df[df['price_range']>=0]

# MLPClassifier 

<font size=4><mark> На початку я захотіла спробувати MLPClassifier, я думаю, що тут підійде правило: "проста модель для складного<br><br> завдання". Вхідних параметрів у нас досить багато, я інтуїтивно сказати не можу, яка має бути цінова категорія,<br><br> значить для моделі це має бути легко. Використовую ті самі стовпчики, які добре корелюються з ціною, але<br><br> слабо між собою.

In [34]:
from sklearn.model_selection import train_test_split

X = df_full.drop(labels = ['weight', 'dual_sim', 'wifi','price_range'], axis=1)
Y = df_full['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 20)

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
from sklearn.neural_network import MLPClassifier

In [55]:
mlp = MLPClassifier(max_iter = 1000, activation = 'logistic') #`relu`, `logistic', 'identity', 'tanh'

In [56]:
mlp.fit(X_train, Y_train)

/Users/hanna/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', max_iter=1000)

In [57]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred)/len(y_true)
    return accuracy

In [58]:
pred = mlp.predict(X_test)
accuracy(Y_test, pred)

0.9645522388059702

In [59]:
res = "{:<10} | {:<10}".format("Y_test", "Y_pred\n")
res += "-"*65+"\n"
res += "\n".join("{:<10} | {:<10}".format(x, y) for x, y in zip(Y_test, pred))
res += "\n"+"-"*65+"\n"
print(res)

Y_test     | Y_pred
   -----------------------------------------------------------------
1          | 1.0       
3          | 3.0       
3          | 3.0       
0          | 0.0       
0          | 0.0       
2          | 2.0       
0          | 0.0       
2          | 2.0       
3          | 3.0       
3          | 3.0       
1          | 1.0       
0          | 0.0       
3          | 3.0       
1          | 1.0       
1          | 1.0       
0          | 0.0       
1          | 1.0       
3          | 3.0       
3          | 3.0       
0          | 0.0       
2          | 2.0       
1          | 1.0       
1          | 1.0       
3          | 3.0       
3          | 3.0       
0          | 0.0       
1          | 1.0       
3          | 3.0       
0          | 0.0       
3          | 3.0       
2          | 2.0       
1          | 1.0       
1          | 1.0       
1          | 1.0       
2          | 1.0       
0          | 0.0       
1          | 1.0       
1          | 1.0       

<font size=4><mark>Обов'язково заскейлила дані, оскільки вони у нас різних масштабів. І підбираючи різні функції активації прийшла до результату точності 96%.

# Sequential model

In [37]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('phone_float.csv')
df.head()

,Unnamed: 0,battery_power,bluetooth,weight,memory,n_cores,ram,dual_sim,wifi,pixel_height,pixel_width,price_range
0,0,1021.0,1.0,136,53.0,3.0,2631.0,1,0,905.0,1988.0,2.0
1,1,563.0,1.0,145,41.0,5.0,2603.0,1,0,1263.0,1716.0,2.0
2,2,615.0,1.0,131,10.0,6.0,2769.0,0,0,1216.0,1786.0,2.0
3,3,1821.0,1.0,141,44.0,2.0,1411.0,0,0,1208.0,1212.0,1.0
4,4,1859.0,0.0,164,22.0,1.0,1067.0,1,0,1004.0,1654.0,1.0


<font size=4><mark>Оскільки модель не хотіла працювати з типом даних int, я перевела потрібні мені стовпці у формат float і<br><br> зберегла як окремий датафрейм, просто мені так було зручніше працювати в різних ноутбуках паралельно.

In [6]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

In [8]:
model = keras.Sequential()

input_layer = keras.layers.Dense(4, input_shape=[7], activation='sigmoid')
model.add(input_layer)

hidden_layer_1 = keras.layers.Dense(16, activation = 'relu')
model.add(hidden_layer_1)

hidden_layer_2 = keras.layers.Dense(32, activation = 'relu')
model.add(hidden_layer_2)

hidden_layer_3 = keras.layers.Dense(64, activation = 'relu')
model.add(hidden_layer_3)

hidden_layer_4 = keras.layers.Dense(128, activation = 'relu')
model.add(hidden_layer_4)

hidden_layer_5 = keras.layers.Dense(256, activation = 'relu')
model.add(hidden_layer_5)

output_layer = keras.layers.Dense(1, activation='softmax')
model.add(output_layer)

In [9]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy']) # mean_squared_error
# categorical_crossentropy

In [11]:
from sklearn.model_selection import train_test_split
X = df.drop(labels = ['weight', 'dual_sim', 'wifi','price_range'], axis=1)
Y = df['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 20)

In [12]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
smote = SMOTE(random_state=42)
X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test = scaler.transform(X_test)

In [21]:
history = model.fit(X_train_smote, Y_train_smote,
                   epochs=500,
                   steps_per_epoch = 10,
                   validation_split = 0.25,
                   batch_size = 40,
                   callbacks = [early_stopping])

Epoch 1/500
10/10 [==============================] - 0s 5ms/step - loss: 0.9600 - accuracy: 0.3325 - val_loss: 3.1461 - val_accuracy: 0.0000e+00
Epoch 2/500
10/10 [==============================] - 0s 4ms/step - loss: 0.8850 - accuracy: 0.3625 - val_loss: 3.1461 - val_accuracy: 0.0000e+00


In [14]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Test score: ', loss)
print('Test accuracy: ', accuracy)

17/17 [==============================] - 0s 601us/step - loss: 0.9869 - accuracy: 0.5000
Test score:  0.986940324306488
Test accuracy:  0.5


<font size=4><mark>Скільки б я не гралася з даними, я не змогла отримати accuracy вище 50%. 
Спочатку я думала, що це через <br><br> дисбаланс у даних, я спробувала кілька oversampling, як-от: SMOTE, RandomOverSampler (я розглядала саме <br><br> over варіанти, тому що даних отже мало для нейромережі), але краще не стало... <br><br>Тому я вирішила піти іншим шляхом і використати модель, яка добре працює з дисбалансом даних.

# RandomForestClassifier

In [77]:
from sklearn.ensemble import RandomForestClassifier

In [78]:
X = df.drop(labels = ['weight', 'dual_sim', 'wifi','price_range'], axis=1)
Y = df['price_range']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [79]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [80]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [81]:
y_pred = rf.predict(X_test)

In [82]:
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9477611940298507


<font size=4><mark> Навіть сумно, що без особливих зусиль і додаткових параметрів отримуємо точність 95%